In [1]:
'''
1. データセットの読み込みと正規化
'''
# tensorflow.keras のインポート
from tensorflow import keras

# Fashion-MNISTデータセットの読み込み
(x_train, t_train), (x_test, t_test) = keras.datasets.fashion_mnist.load_data()

# 訓練データを正規化
x_train = x_train.astype('float32') / 255
# テストデータを正規化
x_test =  x_test.astype('float32') / 255

# (60000, 28, 28)の3階テンソルを(60000, 28, 28, 1)の4階テンソルに変換
x_train = x_train.reshape(-1, 28, 28, 1)
# (10000, 28, 28)の3階テンソルを(10000, 28, 28, 1)の4階テンソルに変換
x_test = x_test.reshape(-1, 28, 28, 1)

In [2]:
'''
2.モデルの作成
'''
class CNN(keras.Model):
    '''畳み込みニューラルネットワーク
    
    Attributes:
      c1(Conv2D): 畳み込み層1
      c2(Conv2D): 畳み込み層2
      p1(MaxPooling2D): プーリング層1
      c3(Conv2D): 畳み込み層3
      p2(MaxPooling2D): プーリング層2
      d1(Dropout): ドロップアウト1
      f1(Flatten): Flatten
      l1(Dense): 全結合層
      l2(Dense): 出力層
    '''
    def __init__(self, output_dim):
        '''
        Parameters:
          output_dim(int): 出力層のユニット数(次元)
                  '''
        super().__init__()
        
        # 正則化の係数
        weight_decay = 1e-4

        # （第1層）畳み込み層1
        # ニューロン数：64
        # 出力：1ニューロンあたり(28, 28, 1)の3階テンソルを64出力するので
        #       (28, 28, 64)の3出力となる
        self.c1 = keras.layers.Conv2D(
            filters=64,                   # フィルター数64
            kernel_size=(3, 3),           # 3×3のフィルター
            padding='same',               # ゼロパディング
            input_shape=x_train[0].shape, # 入力データの形状
            kernel_regularizer=keras.regularizers.l2(
                weight_decay), # 正則化
            activation='relu'             # 活性化関数はReLU
            )
        
        # （第2層）畳み込み層2
        # ニューロン数：32
        # 出力：1ニューロンあたり(28, 28, 1)の3階テンソルを32出力するので
        #       (28, 28, 32)の出力となる
        self.c2 = keras.layers.Conv2D(
            filters=32,                   # フィルターの数は32
            kernel_size=(3, 3),           # 3×3のフィルターを使用
            padding='same',               # ゼロパディングを行う
            kernel_regularizer=keras.regularizers.l2(
                weight_decay),           # 正則化
            activation='relu'             # 活性化関数はReLU
            )
        
        # （第3層）プーリング層1
        # ニューロン数：32
        # 出力：1ニューロンあたり(14, 14, 1)の3階テンソルを32出力するので
        #       (14, 14, 32)の出力となる
        self.p1 = keras.layers.MaxPooling2D(
            pool_size=(2, 2))             # 縮小対象の領域は2×2
        
        # （第4層）畳み込み層3
        # ニューロン数：16
        # 出力：1ニューロンあたり(14 14 1)の3階テンソルを16出力するので
        #       (14, 14, 16)の出力となる
        self.c3 = keras.layers.Conv2D(
            filters=16,                   # フィルターの数は16
            kernel_size=(3, 3),           # 3×3のフィルターを使用
            padding='same',               # ゼロパディングを行う
            kernel_regularizer=keras.regularizers.l2(weight_decacy), # 正則化
            activation='relu'             # 活性化関数はReLU
            )
        
        # （第5層）プーリング層2
        # ニューロン数：16
        # 出力：1ニューロンあたり(7, 7, 1)の3階テンソルを16出力するので
        #       (7, 7, 16)の出力となる
        self.p2 = keras.layers.MaxPooling2D(
            pool_size=(2, 2))             # 縮小対象の領域は2×2

        # ドロップアウト40％
        self.d1 = keras.layers.Dropout(0.4)
        
        # Flaten
        # ニューロン数＝7×7×16=784
        # (7, 7, 64)を(784)にフラット化
        self.f1 = keras.layers.Flatten()

        # （第6層）全結合層
        # ニューロン数：128
        # 出力：(128)の1階テンソルを出力
        self.l1 =  keras.layers.Dense(
            128,                          # ニューロン数は128
            activation='relu')            # 活性化関数はReLU

        # （第7層）出力層
        # ニューロン数：10
        # 出力：要素数(10)の1階テンソルを出力
        self.l2 =  keras.layers.Dense(
            output_dim,                   # 出力層のニューロン数は10
            activation='softmax')         # 活性化関数はソフトマックス
        
        # すべての層をリストにする
        self.ls = [self.c1, self.c2, self.p1, self.c3,
                   self.p2, self.d1, self.f1, self.l1, self.l2]

    def call(self, x):
        '''MLPのインスタンスからコールバックされる関数
        
        Parameters: x(ndarray(float32)):訓練データ、または検証データ
        Returns(float32): CNNの出力として要素数10の1階テンソル        
        '''
        for layer in self.ls:
            x = layer(x)
        
        return x

In [3]:
'''
3.損失関数の定義
'''
# 損失関数はスパースラベル対応クロスエントロピー誤差
cce = keras.losses.SparseCategoricalCrossentropy()

def loss(t, y):
    '''損失関数
    Parameters: t(ndarray(float32)):正解ラベル
                y(ndarray(float32)):予測値
                
    Returns: クロスエントロピー誤差
    '''
    return cce(t, y)

In [4]:
'''
4.オプティマイザーのオブジェクトの生成
  損失と精度を測定するオブジェクトの生成
'''
# 勾配降下アルゴリズムを使用するオプティマイザーを生成
optimizer = keras.optimizers.SGD(learning_rate=0.1)

# 損失を記録するオブジェクトを生成
train_loss = keras.metrics.Mean()
# 精度を記録するオブジェクトを生成
train_acc = keras.metrics.SparseCategoricalAccuracy()
# 検証時の損失を記録するオブジェクトを生成
val_loss = keras.metrics.Mean()
# 検証時の精度を記録するオブジェクトを生成
val_acc = keras.metrics.SparseCategoricalAccuracy()

In [5]:
'''
5.勾配降下アルゴリズムによるパラメーターの更新処理
'''
import tensorflow as tf

def train_step(x, t):
    '''学習を1回行う
    
    Parameters: x(ndarray(float32)):訓練データ
                t(ndarray(float32)):正解ラベル
                
    Returns:
      ステップごとのクロスエントロピー誤差
    '''
    # 自動微分による勾配計算を記録するブロック
    with tf.GradientTape() as tape:
        # モデルに入力して順伝搬の出力値を取得
        outputs = model(x)
        # 出力値と正解ラベルの誤差
        tmp_loss = loss(t, outputs)
        
    # tapeに記録された操作を使用して誤差の勾配を計算        
    grads = tape.gradient(
        # 現在のステップの誤差
        tmp_loss,
        # バイアス、重みのリストを取得
        model.trainable_variables)
    # 勾配降下法の更新式を適用してバイアス、重みを更新
    optimizer.apply_gradients(zip(grads,
                                  model.trainable_variables))
    
    # 損失をMeanオブジェクトに記録
    train_loss(tmp_loss)
    # 精度をCategoricalAccuracyオブジェクトに記録
    train_acc(t, outputs)

In [6]:
'''
6. 検証データによる評価を行う関数
'''
def val_step(x, t):
    '''検証データをモデルに入力して損失と精度を測定
    
    Parameters: x(ndarray(float32)):検証データ
                t(ndarray(float32)):正解ラベル
    '''
    # 検証データの予測値を取得
    preds = model(x)
    # 出力値と正解ラベルの誤差
    tmp_loss = loss(t, preds)
    # 損失をMeanオブジェクトに記録
    val_loss(tmp_loss)
    # 精度をSpase_CategoricalAccuracyオブジェクトに記録
    val_acc(t, preds)

In [7]:
'''
7. 訓練データの20パーセントのデータを検証用のデータにする
'''
from sklearn.model_selection import train_test_split

# 訓練データと検証データに8：2の割合で分割  \は行継続文字
x_train, x_val, t_train, t_val = \
    train_test_split(x_train, t_train, test_size=0.2)

In [8]:
%%time
'''
8.モデルを生成して学習する
'''
from sklearn.utils import shuffle

# エポック数
epochs = 100
# ミニバッチのサイズ
batch_size = 64
# 訓練データのステップ数
steps = x_train.shape[0] // batch_size
# 検証データのステップ数
steps_val = x_val.shape[0] // batch_size

# 出力層10ニューロンのモデルを生成
model = CNN(10)

# 学習を行う
for epoch in range(epochs):
    # 訓練データと正解ラベルをシャッフル
    x_, t_ = shuffle(x_train, t_train, random_state=1)
    
    # 1ステップにおけるミニバッチを使用した学習
    for step in range(steps):
        start = step * batch_size # ミニバッチの先頭インデックス
        end = start + batch_size  # ミニバッチの末尾のインデックス
        # ミニバッチでバイアス、重みを更新
        train_step(x_[start:end], t_[start:end])
        
    # 検証データによるモデルの評価
    for step_val in range(steps_val):
        start = step_val * batch_size # ミニバッチの先頭インデックス
        end = start + batch_size      # ミニバッチの末尾のインデックス
        # 検証データのミニバッチで損失と精度を測定
        val_step(x_[start:end], t_[start:end])

    # 1エポックごとに結果を出力
    print('epoch({}) train_loss: {:.4} train_acc: {:.4}'
          'val_loss: {:.4} val_acc: {:.4}'.format(
              epoch+1,
              train_loss.result(), # 訓練データの損失を出力
              train_acc.result(),  # 訓練データの精度を出力
              val_loss.result(),   # 検証データの損失を出力
              val_acc.result()     # 検証データの精度を出力
              ))

epoch(1) train_loss: 0.6147 train_acc: 0.7725val_loss: 0.5319 val_acc: 0.7932
epoch(2) train_loss: 0.4904 train_acc: 0.8194val_loss: 0.4601 val_acc: 0.8226
epoch(3) train_loss: 0.4296 train_acc: 0.8421val_loss: 0.4066 val_acc: 0.8453
epoch(4) train_loss: 0.3903 train_acc: 0.8567val_loss: 0.3678 val_acc: 0.8612
epoch(5) train_loss: 0.3618 train_acc: 0.8671val_loss: 0.3402 val_acc: 0.872
epoch(6) train_loss: 0.3395 train_acc: 0.8752val_loss: 0.3188 val_acc: 0.8802
epoch(7) train_loss: 0.3212 train_acc: 0.882val_loss: 0.301 val_acc: 0.887
epoch(8) train_loss: 0.3057 train_acc: 0.8877val_loss: 0.2862 val_acc: 0.8926
epoch(9) train_loss: 0.2921 train_acc: 0.8928val_loss: 0.2737 val_acc: 0.8972
epoch(10) train_loss: 0.2798 train_acc: 0.8973val_loss: 0.263 val_acc: 0.9011
epoch(11) train_loss: 0.2686 train_acc: 0.9015val_loss: 0.2539 val_acc: 0.9044
epoch(12) train_loss: 0.2582 train_acc: 0.9054val_loss: 0.2453 val_acc: 0.9076
epoch(13) train_loss: 0.2485 train_acc: 0.909val_loss: 0.2377 val_

In [12]:
'''
9. テストデータによるモデルの評価
'''
# 損失を記録するオブジェクトを生成
test_loss = keras.metrics.Mean()
# 精度を記録するオブジェクトを生成
test_acc = keras.metrics.SparseCategoricalAccuracy()

def test_step(x, t):
    '''テストデータをモデルに入力して損失と精度を測定
    
    Parameters: x(ndarray(float32)):テストデータ
                t(ndarray(float32)):正解ラベル
    '''
    # テストデータの予測値を取得
    preds = model(x)
    # 出力値と正解ラベルの誤差
    tmp_loss = loss(t, preds)
    # 損失をMeanオブジェクトに記録
    test_loss(tmp_loss)
    # 精度をSpase_CategoricalAccuracyオブジェクトに記録
    test_acc(t, preds)

# テストデータで予測して損失と精度を取得
test_step(x_test, t_test)

print('test_loss: {:.4f}, test_acc: {:.4f}'.format(
    test_loss.result(),
    test_acc.result()
))

test_loss: 0.8189, test_acc: 0.9111
